In [ ]:
%run "0_Workspace_setup.ipynb"

from NHM_helpers.sf_data_retrieval import (
    create_nwis_sf_df,
    create_OR_sf_df,
    create_ecy_sf_df,
    create_sf_efc_df,
)

from NHM_helpers.NHM_hydrofabric import (
    create_hru_gdf,
    create_poi_df,
    create_default_gages_file,
    read_gages_file,
)

from NHM_helpers.efc import plot_efc, plot_high_low

# Introduction

Critical in evaluation of the NHM model simulated flows is comparison to observed flows in the watershed. This notebook retrieves available streamflow observations from NWIS and two state agencies, the Oregon Water Resources Deparment (OWRD) and the Washington Departent of Ecology (ECY), combines these data sets into one streamflow obervations file and streanflow gage information and metadata, and write the database out as a netCDF file (.nc) to be used in Notebook "6_Streamflow_Output_Visualization" and other notebooks in NHM-Assist. A complete database of streamflow gages and observation in the model domain is nessessary to evaluate the NHM model and identify other gages that could be included in a model recalibration to improve the model performance.

Three key facts about streamflow observations and the NHM must be reviewed.
- Streamflow observations are NOT used when running PRMS or pywatershed. These data are meant for comparison of simulated output only.
- The NHM DOES use streamflow observations from NWIS in the model calibration workflow (not the streamflow file).
- Limited streamflow gage information is stored in the parameter file.

The paramter file has  are dimensioned by npoigages and include :
- poi_gage_id, the agency identification number
- poi_gage_segment, model segment identification number (nhm_seg) on which the gage falls (1 gage/segment only),
- poi_type, historically used, but not currently used.

It is important to note that the gages in the parameter file are NOT a complete set of gages in the model domain, and NOT all used to calibrate the model. 

## Load NHM domain hydrofabric elements: hrus and segments.

In [ ]:
hru_gdf, hru_txt, hru_cal_level_txt = create_hru_gdf(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
)

poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    gages_file,
    default_gages_file,
    nwis_gage_nobs_min,
)

## Retrieve all NWIS gage information and streamflow observations.
This function pulls time series data for all NWIS gages in the domain, and then filters data to the simulation period (nwis_gages_cache.nc), and creates NWISgages.csv. Both the time series data file and the NWISgages.csv contain all site information for gages with a period of record greater than specified threshold (nwis_gage_nobs_min in nb 0) within the simulation period **AND** gages in the parameter file that have a period of record less than specified threshold.

In [ ]:
NWIS_df = create_nwis_sf_df(
    control_file_name,
    model_dir,
    output_netcdf_filename,
    hru_gdf,
    poi_df,
    nwis_gage_nobs_min,
)

## Make the default gages file (default_gages.csv)
Read in the poi_df (gages in the parameter file) and gages_df (gages in the parameter file **AND** additional NWIS gages or user added gages in the domain.

In [ ]:
default_gages_file = create_default_gages_file(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
    poi_df,
)

gages_df, gages_txt, gages_txt_nb2 = read_gages_file(
    model_dir,
    poi_df,
    gages_file,
)

In [ ]:
default_gages_file

In [ ]:
# Double check this; we think we moved this to the functions

crs = 4326

# Make a list if the HUC2 region(s) the subbasin intersects for NWIS queries
huc2_gdf = gpd.read_file("./data_dependencies/HUC2/HUC2.shp").to_crs(crs)
model_domain_regions = list((huc2_gdf.clip(hru_gdf).loc[:]["huc2"]).values)

# Retrieve daily streamflow data for gages in the gages file
##### We will not use the list of gages in the model paramter file, but will use the gages lsited in the gages_file.csv. This will be useful later for adding gages to the model for further validation or calibration. Also, multiple gages cannot be associated with a single segment outflow in the parameter file. We want a streamflow data set that is more inclusive.

## Retrieve available daily streamflow data from Oregon Water Resources Department
#### https://apps.wrd.state.or.us/apps/sw/hydro_near_real_time/

In [ ]:
owrd_df = create_OR_sf_df(control, model_dir, output_netcdf_filename, hru_gdf, gages_df)

## Retrieve available daily streamflow data from Washing Department of Ecology
#### https://waecy.maps.arcgis.com/apps/Viewer/index.html?appid=832e254169e640fba6e117780e137e7b

In [ ]:
ecy_df = create_ecy_sf_df(control, model_dir, output_netcdf_filename, hru_gdf, gages_df)

## Retrieve available daily streamflow data from NWIS
#### https://waterdata.usgs.gov/nwis/sw

In [ ]:
NWIS_df = create_nwis_sf_df(
    control_file_name,
    model_dir,
    output_netcdf_filename,
    hru_gdf,
    poi_df,
    nwis_gage_nobs_min,
)

# Create streamflow observations file with appended EFC values (sf_efc.nc).

In [ ]:
xr_streamflow = create_sf_efc_df(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
    output_netcdf_filename,
    owrd_df,
    ecy_df,
    NWIS_df,
    gages_df,
)

## Check streamflow observations file: plot discharge and efc information for a selected gage.

In [ ]:
cpoi_id = xr_streamflow.poi_id.values[2]
print(
    f"Quick plots below for gage: {cpoi_id}; Some gages may show no data because some gages in the paramter file have data outside the simulation period."
)
start_date = pd.to_datetime(str(control.start_time)).strftime("%m/%d/%Y")
end_date = pd.to_datetime(str(control.end_time)).strftime("%m/%d/%Y")
ds_sub = xr_streamflow.sel(poi_id=cpoi_id, time=slice(start_date, end_date))
ds_sub = ds_sub.to_dataframe()
flow_col = "discharge"

In [ ]:
plot_efc(ds_sub, flow_col)

In [ ]:
plot_high_low(ds_sub, flow_col)